<a href="https://colab.research.google.com/github/UCAS-BigBird/-123/blob/master/%E5%8F%88%E5%BC%80%E5%A7%8B%E4%BA%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import  roc_auc_score
from timeit import  default_timer as timer
from sklearn.metrics import  roc_curve
MAX_EVALS = 500
N_FOLDS=10

In [0]:
data=pd.read_csv('caravan-insurance-challenge.csv')
train = data[data['ORIGIN'] == 'train']
test = data[data['ORIGIN'] == 'test']

# Extract the labels and format properly
train_labels = np.array(train['CARAVAN'].astype(np.int32)).reshape((-1,))
test_labels = np.array(test['CARAVAN'].astype(np.int32)).reshape((-1,))

# Drop the unneeded columns
train = train.drop(columns = ['ORIGIN', 'CARAVAN'])
test = test.drop(columns = ['ORIGIN', 'CARAVAN'])
#for splitting in cross validation
features = np.array(train)
test_features = np.array(test)
#取所有的train——label
labels = train_labels[:]
print('Train shape: ', train.shape)
print('Test shape: ', test.shape)
train = data[data['ORIGIN'] == 'train']
test = data[data['ORIGIN'] == 'test']

# Extract the labels and format properly
train_labels = np.array(train['CARAVAN'].astype(np.int32)).reshape((-1,))
test_labels = np.array(test['CARAVAN'].astype(np.int32)).reshape((-1,))

# Drop the unneeded columns
train = train.drop(columns = ['ORIGIN', 'CARAVAN'])
test = test.drop(columns = ['ORIGIN', 'CARAVAN'])
#for splitting in cross validation
features = np.array(train)
test_features = np.array(test)
#取所有的train——label
labels = train_labels[:]
print('Train shape: ', train.shape)
print('Test shape: ', test.shape)

Train shape:  (5822, 85)
Test shape:  (4000, 85)
Train shape:  (5822, 85)
Test shape:  (4000, 85)


In [0]:
import random
# Hyperparameter grid
param_grid = {
    'class_weight': [None, 'balanced'],
    'boosting_type': ['gbdt', 'goss', 'dart'],
    'num_leaves': list(range(30, 150)),
    'learning_rate': list(np.logspace(np.log(0.005), np.log(0.2), base = np.exp(1), num = 1000)),
    'subsample_for_bin': list(range(20000, 300000, 20000)),
    'min_child_samples': list(range(20, 500, 5)),
    'reg_alpha': list(np.linspace(0, 1)),
    'reg_lambda': list(np.linspace(0, 1)),
    'colsample_bytree': list(np.linspace(0.6, 1, 10))
}
subsample_dist = list(np.linspace(0.5, 1, 100))

In [0]:
random_results = pd.DataFrame(columns = ['loss', 'params', 'iteration', 'estimators', 'time'],
                       index = list(range(MAX_EVALS)))
train_set = lgb.Dataset(train, label = train_labels)

def random_objective(params, iteration, n_folds = N_FOLDS):
    """Random search objective function. Takes in hyperparameters
       and returns a list of results to be saved."""
    start = timer()
    # Perform n_folds cross validation
    cv_results = lgb.cv(params, train_set, num_boost_round = 10000, nfold = n_folds, 
                        early_stopping_rounds = 100, metrics = 'auc', seed = 50)
    end = timer()
    best_score = np.max(cv_results['auc-mean'])
    # Loss must be minimized
    loss = 1 - best_score
    # Boosting rounds that returned the highest cv score
    n_estimators = int(np.argmax(cv_results['auc-mean']) + 1)
    # Return list of results
    return [loss, params, iteration, n_estimators, end - start]

for i in range(MAX_EVALS):
    params = {key: random.sample(value, 1)[0] for key, value in param_grid.items()}
    print(params)
    if params['boosting_type'] == 'goss':
        # Cannot subsample with goss
        params['subsample'] = 1.0
    else:
        # Subsample supported for gdbt and dart
        params['subsample'] = random.sample(subsample_dist, 1)[0]    
    results_list = random_objective(params, i)
    # Add results to next row in dataframe
    random_results.loc[i, :] = results_list
    
random_results.to_csv('final_result.csv')

{'class_weight': 'balanced', 'boosting_type': 'dart', 'num_leaves': 145, 'learning_rate': 0.031332196449772066, 'subsample_for_bin': 180000, 'min_child_samples': 230, 'reg_alpha': 0.2857142857142857, 'reg_lambda': 0.8163265306122448, 'colsample_bytree': 0.8222222222222222}


/usr/local/lib/python3.6/dist-packages/lightgbm/callback.py:189: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')
